In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import os
from google.colab import drive
drive.mount('/content/drive')
dir = "/content/drive/My Drive/misinformation-NLP/"
os.listdir(dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['yake_reddit.csv',
 'ILM_fake_news_token_72hours',
 'CORD19-GPT',
 'scifact',
 'reddit_BERT_SCIFACT.csv',
 '.ipynb_checkpoints',
 'results',
 'fake-news-corpus',
 'reddit-data',
 'masked_claims',
 'models',
 'tmp',
 'data_for_github',
 'AMT_evidence_batches']

In [ ]:
claims = pd.read_csv('/content/drive/MyDrive/misinformation-NLP/reddit-data/pushshift_additional_1k.csv')[['title', 'flair', 'source_url']]
claims['claim'] = claims['title'].apply(lambda x: " ".join(x.split(" ")))
claims.drop_duplicates(subset=['claim'], inplace=True)
claims

,title,flair,source_url,claim
0,Air Pollution Is Associated with COVID-19 Inci...,General,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Air Pollution Is Associated with COVID-19 Inci...
1,Fauci says Trump policies very likely led to d...,General,https://www.scmp.com/news/world/united-states-...,Fauci says Trump policies very likely led to d...
2,Boris Johnson says U.K. coronavirus variant ma...,Press Release,https://www.washingtonpost.com/world/europe/uk...,Boris Johnson says U.K. coronavirus variant ma...
3,New Uk variant may be more deadly,Press Release,https://www.bbc.co.uk/news/health-55768627,New Uk variant may be more deadly
4,New Uk variant May be more deadly,General,https://www.bbc.co.uk/news/health-55768627l,New Uk variant May be more deadly
...,...,...,...,...
550,42 confirmed cases of coronavirus in India ; I...,General,https://www.emedinexus.com/post/16824/?utm_sou...,42 confirmed cases of coronavirus in India ; I...
551,Canada 's Military Ordered to Begin Preparing ...,Government Agency,https://www.7ummitmagazine.com/latest-news/202...,Canada 's Military Ordered to Begin Preparing ...
552,CDC Released Patient Who Tested Positive for C...,Social Impact,https://time.com/5793763/san-antonio-covid-19/,CDC Released Patient Who Tested Positive for C...
553,Iran says coronavirus has spread to several ci...,Government Agency,https://edition.cnn.com/asia/live-news/coronav...,Iran says coronavirus has spread to several ci...


##GENERATING KEYWORDS

In [ ]:
!!pip install --upgrade transformers
!!git clone https://github.com/huggingface/transformers
# !!pip install ./transformers
!!pip install -r ./transformers/examples/requirements.txt
!!pip install boto3
!!pip install --upgrade pyarrow
!!pip install --upgrade urllib3
!!pip install --upgrade chardet
import sys
sys.path.insert(1, '/content/transformer-drg-style-transfer')
!!git clone https://github.com/agaralabs/transformer-drg-style-transfer
import csv
import logging
import os
import random
import sys
import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange


from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
#from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear

from bertviz.bertviz import attention, visualization
from bertviz.bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)

# UNCASED MODEL
logger = logging.getLogger(__name__)

bert_classifier_model_dir =  '/content/drive/MyDrive/misinformation-NLP/models/BERT-SCIFACT-UNCASED' #"/content/BERT-SCIFACT-UNCASED" ## Path of BERT classifier model path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
logger.info("device: {}, n_gpu {}".format(device, n_gpu))

## Model for performing Classification
model_cls = BertForSequenceClassification.from_pretrained(bert_classifier_model_dir, num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # replace scibert ?
model_cls.to(device)
model_cls.eval()

## Model to get the attention weights of all the heads
model = BertModel.from_pretrained(bert_classifier_model_dir)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model.to(device)
model.eval()


common_words=['is','are','was','were','has','have','had','a','an','the','this','that','these','those','there','how','i','we',
             'he','she','it','they','them','their','his','him','her','us','our', 'and','in','my','your','you', 'will', 'shall']
common_words_tokens = tokenizer.convert_tokens_to_ids(common_words)
not_to_remove_ids = tokenizer.convert_tokens_to_ids(["[CLS]","[SEP]", ".", "?", "!"])
not_to_remove_ids += common_words_tokens

max_seq_len=70 # Maximum sequence length 
sm = torch.nn.Softmax(dim=-1) ## Softmax over the batch
def run_attn_examples(input_sentences, layer, head, bs=128):
    """
    Returns Attention weights for selected Layer and Head along with ids and tokens
    of the input_sentence
    """
    ids = []
    ids_to_decode = [None for k in range(len(input_sentences))]
    tokens_to_decode = [None for k in range(len(input_sentences))]
    segment_ids = []
    input_masks = []
    attention_weights = [None for z in input_sentences]
    ## BERT pre-processing
    for j,sen in enumerate(tqdm(input_sentences)):
        
        text_tokens = tokenizer.tokenize(sen)
        if len(text_tokens) >= max_seq_len-2:
            text_tokens = text_tokens[:max_seq_len-4]
        tokens = ["[CLS]"] + text_tokens + ["[SEP]"]
        tokens_to_decode[j] = tokens
        temp_ids = tokenizer.convert_tokens_to_ids(tokens)
        ids_to_decode[j] = temp_ids
        input_mask = [1] * len(temp_ids)
        segment_id = [0] * len(temp_ids)
        padding = [0] * (max_seq_len - len(temp_ids))
        
        temp_ids += padding
        input_mask += padding
        segment_id += padding
        
        ids.append(temp_ids)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    
    # Convert Ids to Torch Tensors
    ids = torch.tensor(ids) 
    segment_ids = torch.tensor(segment_ids)
    input_masks = torch.tensor(input_masks)
    
    steps = len(ids) // bs
    
    for i in trange(steps+1):
        if i == steps:
            temp_ids = ids[i * bs : len(ids)]
            temp_segment_ids = segment_ids[i * bs: len(ids)]
            temp_input_masks = input_masks[i * bs: len(ids)]
        else:
            temp_ids = ids[i * bs : i * bs + bs]
            temp_segment_ids = segment_ids[i * bs: i * bs + bs]
            temp_input_masks = input_masks[i * bs: i * bs + bs]
        
        temp_ids = temp_ids.to(device)
        temp_segment_ids = temp_segment_ids.to(device)
        temp_input_masks = temp_input_masks.to(device)
        with torch.no_grad():
             _, _, attn = model(temp_ids, temp_segment_ids, temp_input_masks)
        # Concate Attention weights
        for j in range(len(attn[layer]['attn_probs'])):
            attention_weights[i * bs + j] = (attn[layer]['attn_probs'][j][head][0]).to('cpu')
    
    return attention_weights, ids_to_decode, tokens_to_decode

def prepare_data(aw, ids_to_decode, tokens_to_decode):
    #out_sen = [None for i in range(len(aw))]
    all_top_words = []
    for i in trange(len(aw)):
        topv, topi = aw[i].topk(len(ids_to_decode[i]))
        #print(ids_to_decode[i].index(0))
        topv, topi = aw[i].topk(ids_to_decode[i].index(0)) #aw[i].topk(k=2)
        #topv, topi = aw[i].topk(k=2)
        topi = topi.tolist()
        topv = topv.tolist()
        #print(topi)
        #print(topv)
        # print(i,train_0[i])
        # print(tokens_to_decode[i])
        # print("Original Top Indexes = {}".format(topi))
        topi = [topi[j] for j in range(len(topi)) if ids_to_decode[i][topi[j]] not in not_to_remove_ids] # remove noun and common words
        #print("After removing Nouns = {}".format(topi))
        topi = [topi[j] for j in range(len(topi)) if "##" not in tokens_to_decode[i][topi[j]]] # Remove half words
        #print("After removing Half-words = {}".format(topi))
        
        top_words = []
        printed=0 
        for j in range(0, len(topi)):
          
          top_word = tokens_to_decode[i][topi[j]]
          if len(top_word) >= 3 and "virus" not in top_word \
             and "corona" not in top_word and printed<3:
            printed+=1
            #print(top_word)
            top_words.append(top_word)
        all_top_words.append(top_words)

    return all_top_words

layer_ = 6
head_ = 8
print()
print(layer_, head_)
print()
train_0 = ["Ending coronavirus lockdowns will be a dangerous process of trial and error", 
           "Antibody tests suggest that coronavirus infections vastly exceed official counts",
           "Study Finds Nearly Everyone Who Recovers From COVID-19 Makes Coronavirus Antibodies",
           "New Zealand eliminates COVID-19",
           "Evidence that Vitamin D Supplementation Could Reduce Risk of Influenza and COVID-19 Infections and Deaths"]
aw, ids_to_decode, tokens_to_decode = run_attn_examples(train_0, layer=layer_, head=head_, bs=128)
top_words = prepare_data(aw, ids_to_decode, tokens_to_decode)
print(top_words)

01/23/2021 03:38:10 - INFO - __main__ -   device: cpu, n_gpu 0
01/23/2021 03:38:10 - INFO - pytorch_pretrained_bert.modeling -   loading archive file /content/drive/MyDrive/misinformation-NLP/models/BERT-SCIFACT-UNCASED
01/23/2021 03:38:10 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

01/23/2021 03:38:14 - INFO - bertviz.bertviz.pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
01/23/2021 03:38:15 - 


6 8



100%|██████████| 5/5 [00:00<00:00, 2605.16it/s]

[['error', 'dangerous', 'ending'], ['counts', 'tests', 'suggest'], ['study', 'finds', 'makes'], ['eliminate', 'zealand', 'new'], ['deaths', 'could', 'reduce']]


In [ ]:
import re

train_0 = [c.replace("\"", "") for c in list(claims['claim'])]
aw, ids_to_decode, tokens_to_decode = run_attn_examples(train_0, layer=layer_, head=head_, bs=128)
top_words = prepare_data(aw, ids_to_decode, tokens_to_decode)

100%|██████████| 552/552 [00:00<00:00, 20996.05it/s]


In [ ]:
outfile = '/content/drive/MyDrive/misinformation-NLP/results/additional_1k.csv'

sources = list(claims['source_url'])
flairs = list(claims['flair'])
with open(outfile, "w") as out_fp:
  writer = csv.writer(out_fp, delimiter=",")
  writer.writerow(['claim', 'keyword', 
                   'src_url', 'flair'])
  for i in range(0, len(top_words)):
    for j in range(0, len(top_words[i])):
      claim = train_0[i]
      keyword = top_words[i][j]
      # mask = claim.lower().replace(keyword, "_")
      # mask = re.sub(r'_\w*', '_', mask)
      # #print(claims[i])
      # #print(claim)
      src_url = sources[i]
      flair = flairs[i]
      writer.writerow([claim, keyword, src_url, flair])

# below is the code to mask the claims for the ILM appraoch
# with open(outfile, "w") as out_fp:
#   writer = csv.writer(out_fp, delimiter=",")
#   writer.writerow(['claim', 'keyword', 'mask', 
#                    'src_url', 'flair'])
#   for i in range(0, len(top_words)):
#     for j in range(0, len(top_words[i])):
#       claim = train_0[i]
#       keyword = top_words[i][j]
#       mask = claim.lower().replace(keyword, "_")
#       mask = re.sub(r'_\w*', '_', mask)
#       #print(claims[i])
#       #print(claim)
#       src_url = sources[i]
#       flair = flairs[i]
#       writer.writerow([claim, keyword,  mask, src_url, flair])

In [ ]:
# Now generate the claims replacing the keywords

In [ ]:
# trying the Roberta pre-trained on CORD19
from transformers import pipeline
from transformers import RobertaTokenizerFast, RobertaForMaskedLM

tokenizer = RobertaTokenizerFast.from_pretrained("amoux/roberta-cord19-1M7k")
model = RobertaForMaskedLM.from_pretrained("amoux/roberta-cord19-1M7k")

fillmask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

text = "Lung infiltrates cause significant morbidity and mortality in immunocompromised patients."
masked_text = text.replace("patients", tokenizer.mask_token)
predictions = fillmask(masked_text, top_k=3)
predictions

[{'score': 0.6273620128631592,
  'sequence': '<s>Lung infiltrates cause significant morbidity and mortality in immunocompromised patients.</s>',
  'token': 660,
  'token_str': 'Ġpatients'},
 {'score': 0.19800478219985962,
  'sequence': '<s>Lung infiltrates cause significant morbidity and mortality in immunocompromised individuals.</s>',
  'token': 1868,
  'token_str': 'Ġindividuals'},
 {'score': 0.022069646045565605,
  'sequence': '<s>Lung infiltrates cause significant morbidity and mortality in immunocompromised animals.</s>',
  'token': 1471,
  'token_str': 'Ġanimals'}]

In [ ]:
text = "Probability of dying of COVID-19 in hospital doubled as L.A. County 's death toll"
masked_text = text.replace("increased", tokenizer.mask_token)
predictions = fillmask(masked_text, top_k=3)
predictions

[{'score': 0.8134004473686218,
  'sequence': "<s>Probability of dying of COVID-19 in hospital doubled as L.A. County's death toll'?</s>",
  'token': 11,
  'token_str': "'"},
 {'score': 0.027315134182572365,
  'sequence': '<s>Probability of dying of COVID-19 in hospital doubled as L.A. County\'s death toll"?</s>',
  'token': 6,
  'token_str': '"'},
 {'score': 0.01859157904982567,
  'sequence': "<s>Probability of dying of COVID-19 in hospital doubled as L.A. County's death toll''?</s>",
  'token': 4149,
  'token_str': "''"}]

In [ ]:
#fname = '/content/drive/MyDrive/misinformation-NLP/tmp/sample.csv'
fname = '/content/drive/MyDrive/misinformation-NLP/results/additional_1k.csv'
claims = pd.read_csv(fname)

#output_fname = "roberta_allValid_attention_reddit.csv"
#output_fname = "roberta_allValid_attention_pushshift10K.csv"
#output_fname = "roberta_PushALLValid_attention.csv"
# output_fname = 'roberta-CORD19-SAMPLE.csv'


# claims = claims[:5]

claims

,claim,keyword,src_url,flair
0,Air Pollution Is Associated with COVID-19 Inci...,with,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,General
1,Air Pollution Is Associated with COVID-19 Inci...,associated,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,General
2,Air Pollution Is Associated with COVID-19 Inci...,austria,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,General
3,Fauci says Trump policies very likely led to d...,likely,https://www.scmp.com/news/world/united-states-...,General
4,Fauci says Trump policies very likely led to d...,deaths,https://www.scmp.com/news/world/united-states-...,General
...,...,...,...,...
1643,Iran says coronavirus has spread to several ci...,says,https://edition.cnn.com/asia/live-news/coronav...,Government Agency
1644,Iran says coronavirus has spread to several ci...,cities,https://edition.cnn.com/asia/live-news/coronav...,Government Agency
1645,China Reports Smallest Number Of New COVID-19 ...,but,https://khn.org/morning-breakout/china-reports...,News Article
1646,China Reports Smallest Number Of New COVID-19 ...,doesn,https://khn.org/morning-breakout/china-reports...,News Article


In [ ]:
dir = '/content/drive/MyDrive/misinformation-NLP/results/'
data = []
i=0
for claim, keyword, source, flair in tqdm(zip(claims['claim'], 
                                                    claims['keyword'], 
                                                    claims['src_url'],
                                                    claims['flair'])):
    i+=1
    if claim[-1] != ".":
      # if there is no . at the end and we are replacing last word, the model will just predict a punctuation mark
      claim = claim +  "."

    masked_text = claim.lower().replace(keyword, tokenizer.mask_token)
    #print(masked_text)

    try:
      suggs = fillmask(masked_text, top_k=10) #CHANGED TO 10
      suggs = [sent['sequence'].replace("<s>", "").replace("</s>", "") for sent in suggs]
      suggs = set(suggs)
    except:
      pass

    for sug in suggs:
        if sug != claim.lower():
          data.append([claim, keyword, sug, source, flair])

    if i%100==0:
        all_fakes = pd.DataFrame(data, columns=['claim', 'keyword', 'false_claim', 'source', 'flair'])
        all_fakes.to_csv(dir+output_fname)

1648it [03:51,  7.11it/s]
